In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
from tqdm import tqdm
import time
import pylab as pl
from IPython import display

In [4]:
def check_dead_end(obs_map, pos, direction):
    x, y = pos
    tunnel=[]
    deadend=0
    limit = obs_map.shape[0]-1
    if direction=='down':
        for i in obs_map[x:,y]:
            tunnel.append(i)
            if i == 1:
                break
    elif direction=='up':
        for i in list(obs_map[:x+1,y])[::-1]:
            tunnel.append(i)
            if i == 1:
                break
    elif direction=='left':
        for i in list(obs_map[x,:y+1])[::-1]:
            tunnel.append(i)
            if i == 1:
                break
    elif direction=='right':
        for i in list(obs_map[x,y:]):
            tunnel.append(i)
            if i == 1:
                break

    if sum(tunnel)==0:
        tunnel.append(1)

    if direction=="down":
        if y==0:
            sidewalk1=list(obs_map[x+1:x+len(tunnel)-1, y+1])
        elif y==limit:
            sidewalk2=list(obs_map[x+1:x+len(tunnel)-1, y-1])
        else:
            sidewalk1=list(obs_map[x+1:x+len(tunnel)-1, y+1])
            sidewalk2=list(obs_map[x+1:x+len(tunnel)-1, y-1])
        check_var=y
    elif direction=="up":
        if y==0:
            sidewalk1=list(obs_map[x-len(tunnel)+2:x, y+1])
        elif y==limit:
            sidewalk2=list(obs_map[x-len(tunnel)+2:x, y-1])
        else:
            sidewalk1=list(obs_map[x-len(tunnel)+2:x, y+1])
            sidewalk2=list(obs_map[x-len(tunnel)+2:x, y-1])
        check_var=y
    elif direction=="right":
        if x==0:
            sidewalk1=list(obs_map[x+1, y+1:y+len(tunnel)-1])
        elif x==limit:
            sidewalk2=list(obs_map[x-1, y+1:y+len(tunnel)-1])
        else:
            sidewalk1=list(obs_map[x+1, y+1:y+len(tunnel)-1])
            sidewalk2=list(obs_map[x-1, y+1:y+len(tunnel)-1])
        check_var=x
    elif direction=="left":
        if x==0:
            sidewalk1=list(obs_map[x+1, y-len(tunnel)+2:y])
        elif x==limit:
            sidewalk2=list(obs_map[x-1, y-len(tunnel)+2:y])
        else:
            sidewalk1=list(obs_map[x+1, y-len(tunnel)+2:y])
            sidewalk2=list(obs_map[x-1, y-len(tunnel)+2:y])
        check_var=x

    if len(tunnel)>2:
        wall = list(np.ones(len(tunnel)-2))
        if check_var==0:
            if sidewalk1==wall:
                deadend=1
        elif check_var==limit:
            if sidewalk2==wall:
                deadend=1
        else:
            if sidewalk2==wall and sidewalk1==wall:
                deadend=1
    else:
        deadend=1

    return deadend

In [5]:
def findPath(grid, obstacles, curPos, T, visited, path, rows, cols):
    visited[curPos[0]][curPos[1]] = True
    path.append(curPos)

    if curPos == T:
        if len(path) == rows * cols - len(obstacles):
            return path
        else:
            visited[curPos[0]][curPos[1]] = False
            path.pop()
            return None

    for dx, dy in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
        nx = curPos[0] + dx
        ny = curPos[1] + dy

        if 0 <= nx < rows and 0 <= ny < cols and not visited[nx][ny] and grid[nx][ny] != 1:
            newPath = findPath(grid, obstacles, (nx, ny), T, visited, path, rows, cols)
            if newPath:
                return newPath

    visited[curPos[0]][curPos[1]] = False
    path.pop()
    return None

In [ ]:
def plot_grid(optimal_path, wait, grid, obstacle_cells, entry_cell, exit_cell):
    fig, ax = plt.subplots()
    grid_size = grid.shape[0]
    ax.set_xticks(np.arange(-0.5, grid_size, 1))
    ax.set_yticks(np.arange(-0.5, grid_size, 1))
    ax.set_xlim((-0.5,grid_size-1+0.5))
    ax.set_ylim((-0.5,grid_size-1+0.5))
    ax.grid(color='k', linestyle='-', linewidth=1)
    ax.imshow(grid, cmap='gray', extent=[-0.5, grid_size+0.5, -0.5, grid_size+0.5])
    ax.add_patch(plt.Rectangle((entry_cell[0]-0.5, entry_cell[1]-0.5), 1, 1, color='green'))
    ax.scatter(entry_cell[0], entry_cell[1], color='g', marker='o', label='Entry', s=40, zorder=6)
    ax.add_patch(plt.Rectangle((exit_cell[0]-0.5, exit_cell[1]-0.5), 1, 1, color='red'))
    ax.scatter(exit_cell[0], exit_cell[1], color='r', marker='o', label='Exit', s=40, zorder=6)

    for obstacle in obstacle_cells:
        ax.add_patch(plt.Rectangle((obstacle[1]-0.5, obstacle[0]-0.5), 1, 1, color='black'))

    for i in range(len(optimal_path)):
        ax.text(optimal_path[i][1], optimal_path[i][0], str(i), color='red', ha='center', va='center', zorder=6)

    # Plot the optimal path
    x_optimal = [cell[1] for cell in optimal_path]
    y_optimal = [cell[0] for cell in optimal_path]
    ax.plot(x_optimal, y_optimal, color='m', linewidth=2, linestyle='dashed', label='Optimal Path')

    for cell in optimal_path:
        ax.add_patch(plt.Rectangle((cell[1]-0.5, cell[0]-0.5), 1, 1, color='blue'))

        # re-drawing the figure
        display.display(pl.gcf())

        # to flush the GUI events
        display.clear_output(wait=True)
        time.sleep(wait)

    ax.legend()
    plt.title("Grid Navigation Problem")
    plt.show()

In [ ]:
def plot_grid(optimal_path, wait, grid, obstacle_cells, entry_cell, exit_cell):
    fig, ax = plt.subplots()
    grid_size = grid.shape[0]
    ax.set_xticks(np.arange(-0.5, grid_size, 1))
    ax.set_yticks(np.arange(-0.5, grid_size, 1))
    ax.set_xlim((-0.5,grid_size-1+0.5))
    ax.set_ylim((-0.5,grid_size-1+0.5))
    ax.grid(color='k', linestyle='-', linewidth=1)
    ax.imshow(grid, cmap='gray', extent=[-0.5, grid_size+0.5, -0.5, grid_size+0.5])
    ax.add_patch(plt.Rectangle((entry_cell[0]-0.5, entry_cell[1]-0.5), 1, 1, color='green'))
    ax.scatter(entry_cell[0], entry_cell[1], color='g', marker='o', label='Entry', s=40, zorder=6)
    ax.add_patch(plt.Rectangle((exit_cell[0]-0.5, exit_cell[1]-0.5), 1, 1, color='red'))
    ax.scatter(exit_cell[0], exit_cell[1], color='r', marker='o', label='Exit', s=40, zorder=6)

    for obstacle in obstacle_cells:
        ax.add_patch(plt.Rectangle((obstacle[1]-0.5, obstacle[0]-0.5), 1, 1, color='black'))

    for i in range(len(optimal_path)):
        ax.text(optimal_path[i][1], optimal_path[i][0], str(i), color='red', ha='center', va='center', zorder=6)

    # Plot the optimal path
    x_optimal = [cell[1] for cell in optimal_path]
    y_optimal = [cell[0] for cell in optimal_path]
    ax.plot(x_optimal, y_optimal, color='m', linewidth=2, linestyle='dashed', label='Optimal Path')

    for cell in optimal_path:
        ax.add_patch(plt.Rectangle((cell[1]-0.5, cell[0]-0.5), 1, 1, color='blue'))

        # re-drawing the figure
        display.display(pl.gcf())

        # to flush the GUI events
        display.clear_output(wait=True)
        time.sleep(wait)

    ax.legend()
    plt.title("Grid Navigation Problem")
    plt.show()

In [ ]:
def get_obs_map(grid_size, obstacle_cells, agent_path):
    # Update state representation to include occupancy information
    obs_map = np.zeros((grid_size, grid_size))  # State representation for each cell
    for i in range(grid_size):
        for j in range(grid_size):
            if (i, j) in obstacle_cells or (i, j) in agent_path:
                obs_map[i, j] = 1  # Mark obstacle & visited cells as occupied
    return obs_map